In [76]:

from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pickle as pkl
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from __future__ import absolute_import, print_function, division



In [77]:
#load traning data

def load_data():
    with open('train.pkl', 'rb') as f:
        return pkl.load(f)


In [78]:
data=load_data()

X = data[0]  #contains   30134 X 3136 -inputs  
Y = data[1]  #contains   30134 X 1 - outputs


# Split the data - i suggest ratio of data train : validation (test) to be 4 : 1 or something similar - imo 
# the bigger sample the lower ratio


# I haven't used whole dataset but w/e it's still enough

xval = X[23000:30000] 
yval = Y[23000:30000]



xtrain = X[0:23000]
ytrain = Y[0:23000]


# Outputs must be vectors not Dx1 arrays

ytrain=ytrain[:,0]
yval=yval[:,0]



# We have outputs as class number but we have to adapt data to our model so we're just creating
# class value (e.g. 11) -----> vector of inputs[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0 .....] 36x1


# Im pretty sure that where is way more efficient and nicer way to do that - mby I'll google it later
yvalBinary = np.zeros(shape=(yval.shape[0], 36))
for i in range(yval.shape[0]):
    yvalBinary[i][yval[i]] = 1

ytrainBinary = np.zeros(shape=(ytrain.shape[0], 36))
for i in range(ytrain.shape[0]):
    ytrainBinary[i][ytrain[i]] = 1



In [87]:
#I think everythink it clear here

model1 = Sequential()
model1.add(Dense(units=90, activation='relu', input_dim=3136))
model1.add(Dense(units=60, activation='relu'))
model1.add(Dense(units=36, activation='softmax'))

model1.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model1.fit(xtrain, ytrainBinary, epochs=120, batch_size=50)


Epoch 1/120
23000/23000 [==============================] - 4s 188us/step - loss: 2.2608 - acc: 0.4098
Epoch 2/120
23000/23000 [==============================] - 3s 126us/step - loss: 1.3046 - acc: 0.6496
Epoch 3/120
23000/23000 [==============================] - 3s 125us/step - loss: 1.0099 - acc: 0.7160
Epoch 4/120
23000/23000 [==============================] - 3s 128us/step - loss: 0.8755 - acc: 0.7477
Epoch 5/120
23000/23000 [==============================] - 3s 123us/step - loss: 0.7905 - acc: 0.7680
Epoch 6/120
23000/23000 [==============================] - 3s 134us/step - loss: 0.7311 - acc: 0.7841
Epoch 7/120
23000/23000 [==============================] - 3s 120us/step - loss: 0.6831 - acc: 0.7977
Epoch 8/120
23000/23000 [==============================] - 3s 118us/step - loss: 0.6453 - acc: 0.8078
Epoch 9/120
23000/23000 [==============================] - 3s 118us/step - loss: 0.6154 - acc: 0.8174
Epoch 10/120
23000/23000 [==============================] - 3s 118us/step - loss: 

23000/23000 [==============================] - 3s 123us/step - loss: 0.0851 - acc: 0.9747
Epoch 81/120
23000/23000 [==============================] - 3s 126us/step - loss: 0.0845 - acc: 0.9750
Epoch 82/120
23000/23000 [==============================] - 3s 124us/step - loss: 0.0817 - acc: 0.9760
Epoch 83/120
23000/23000 [==============================] - 3s 124us/step - loss: 0.0813 - acc: 0.9763
Epoch 84/120
23000/23000 [==============================] - 3s 120us/step - loss: 0.0777 - acc: 0.9772
Epoch 85/120
23000/23000 [==============================] - 3s 121us/step - loss: 0.0779 - acc: 0.9777
Epoch 86/120
23000/23000 [==============================] - 3s 125us/step - loss: 0.0745 - acc: 0.9788
Epoch 87/120
23000/23000 [==============================] - 3s 123us/step - loss: 0.0729 - acc: 0.9788
Epoch 88/120
23000/23000 [==============================] - 3s 123us/step - loss: 0.0730 - acc: 0.9777
Epoch 89/120
23000/23000 [==============================] - 3s 124us/step - loss: 0.06

In [88]:
#Simple check how we're doing with our keras model - remember we're printing number of cases which were wrong
kerasmodelPred=model1.predict(xval)

sum=0
for i in range (xval.shape[0]):
    if(np.argmax(kerasmodelPred[i])!=yval[i]):
        sum+=1
print(sum)

878


In [99]:
#Get all parameters of the model - if u have more layers you've to add variables

w1=np.asarray(model1.layers[0].get_weights()[0])
bias1=model1.layers[0].get_weights()[1]
w2=np.asarray(model1.layers[1].get_weights()[0])
bias2=model1.layers[1].get_weights()[1]
w3=np.asarray(model1.layers[2].get_weights()[0])
bias3=model1.layers[2].get_weights()[1]

In [100]:
#Function used by our model

def relu(x):
    return x * (x > 0)
    
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

    
    
# Here is an original Keras function implementatnion K- keras  T-theano
# As for softmax - don't bother yourself with searching. It it use way to many subfunctions and all in all comes down to C++

def reluT(x, alpha=0):

    if alpha == 0:
        return 0.5 * (x + abs(x))
    else:
        alpha = tensor.as_tensor_variable(alpha)
        f1 = 0.5 * (1 + alpha)
        f2 = 0.5 * (1 - alpha)
        return f1 * x + f2 * abs(x)
    
def reluK(x, alpha=0.):

    x = reluT(x, alpha)
    return x



In [101]:
#Now we are making our forward pass without keras

def predictWithGivenParams(bias1,bias2,bias3,w1,w2,w3,x):
        return relu((relu((relu((x@w1) + bias1)@w2)+bias2)@w3)+bias3)

In [102]:
# Making the same test for matrix model
matrixmodelPred=predictWithGivenParams(bias1,bias2,bias3,w1,w2,w3,xval)

sum=0
for i in range (xval.shape[0]):
    if(np.argmax(matrixmodelPred[i])!=yval[i]):
        sum+=1
print(sum)

878


In [ ]:
#Here is just example how to load weights to another model - you can ignore next 2 cells, just additional content

model2 = Sequential()
model2.add(Dense(units=90, activation='relu', input_dim=3136))
model2.add(Dense(units=36, activation='softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model2.set_weights(model1.get_weights())

In [ ]:
kerasmodel2=model2.predict(xval)

sum=0
for i in range (xval.shape[0]):
    if(np.argmax(kerasmodel2[i])!=yval[i]):
        sum+=1
print(sum)

In [103]:
# Save your parameters as pickle files - Way better that saving to .txt, although you can also do that

with open('w1', 'wb') as fp:
    pkl.dump(w1, fp)
    
with open('w2', 'wb') as fp:
    pkl.dump(w2, fp)
    
with open('w3', 'wb') as fp:
    pkl.dump(w3, fp)

with open('bias1', 'wb') as fp:
    pkl.dump(bias1, fp)
    
with open('bias2', 'wb') as fp:
    pkl.dump(bias2, fp)
    
with open('bias3', 'wb') as fp:
    pkl.dump(bias3, fp)

In [ ]:
# Now you can just load your files and use it with our matrix model so that you won't need to import keras to run your neural network
    with open ('filename.pkl', 'rb') as fp:
        param = pkl.load(fp)

# Mby I'll add example later on but now I cannot do that because of the fact that it's uni assignment and I can't give my whole code